In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, GlobalAveragePooling1D, Bidirectional
from keras.layers import GlobalMaxPooling1D, Input, concatenate, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM, GRU, CuDNNGRU
from keras import callbacks

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
train_df, val_df = train_test_split(train_df, test_size=0.2) # 20%
test_df = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
train_df.head(7)

**Cleaning**

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

## Puncts function
def clean_text(txt):
    txt = str(txt)
    for punct in puncts:
        txt = txt.replace(punct, f' {punct} ')
    return txt

## Replace numbers with '#' function
def clean_numbers(txt):
    txt = re.sub('[0-9]{5,}', '#####', txt)
    txt = re.sub('[0-9]{4}', '####', txt)
    txt = re.sub('[0-9]{3}', '###', txt)
    txt = re.sub('[0-9]{2}', '##', txt)
    return txt

## change shortcuts words to there full form
def correct(txt):
    txt = re.sub("ain't", "is not", txt)
    txt = re.sub("aren't", "are not", txt)
    txt = re.sub("can't", "cannot", txt)
    txt = re.sub("'cause", "because", txt)
    txt = re.sub("could've", "could have", txt)
    txt = re.sub("couldn't", "could not", txt)
    txt = re.sub("didn't", "did not", txt)
    txt = re.sub("doesn't", "does not", txt)
    txt = re.sub("don't", "do not", txt)
    txt = re.sub("hadn't", "had not", txt)
    txt = re.sub("hasn't", "has not", txt)
    txt = re.sub("haven't", "have not", txt)
    txt = re.sub("he'd", "he would", txt)
    txt = re.sub("he'll", "he will", txt)
    txt = re.sub("he's", "he is", txt)
    txt = re.sub("how'd", "how did", txt)
    txt = re.sub("how'd'y", "how do you", txt)
    txt = re.sub("how'll", "how will", txt)
    txt = re.sub("how's", "how is", txt)
    txt = re.sub("I'd", "I would", txt)
    txt = re.sub("I'd've", "I would have", txt)
    txt = re.sub("I'll", "I will", txt)
    txt = re.sub("I'll've", "I will have", txt)
    txt = re.sub("I'm", "I am", txt)
    txt = re.sub("I've", "I have", txt)
    txt = re.sub("i'd", "i would", txt)
    txt = re.sub("i'd've", "i would have", txt)
    txt = re.sub("i'll", "i will", txt)
    txt = re.sub("i'll've", "i will have", txt)
    txt = re.sub("i'm", "i am", txt)
    txt = re.sub("i've", "i have", txt)
    txt = re.sub("it'd", "it would", txt)
    txt = re.sub("it'd've", "it would have", txt)
    txt = re.sub("it'll", "it will", txt)
    txt = re.sub("it'll've", "it will have", txt)
    txt = re.sub("it's", "it is", txt)
    txt = re.sub("let's", "let us", txt)
    txt = re.sub("ma'am", "madam", txt)
    txt = re.sub("mayn't", "may not", txt)
    txt = re.sub("might've", "might have", txt)
    txt = re.sub("mightn't", "might not", txt)
    txt = re.sub("mightn't've", "might not have", txt)
    txt = re.sub("must've", "must have", txt)
    txt = re.sub("mustn't", "must not", txt)
    txt = re.sub("mustn't've", "must not have", txt)
    txt = re.sub("needn't", "need not", txt)
    txt = re.sub("needn't've", "need not have", txt)
    txt = re.sub("o'clock", "of the clock", txt)
    txt = re.sub("oughtn't", "ought not", txt)
    txt = re.sub("oughtn't've", "ought not have", txt)
    txt = re.sub("shan't", "shall not", txt)
    txt = re.sub("sha'n't", "shall not", txt)
    txt = re.sub("shan't've", "shall not have", txt)
    txt = re.sub("she'd", "she would", txt)
    txt = re.sub("she'd've", "she would have", txt)
    txt = re.sub("she'll", "she will", txt)
    txt = re.sub("she'll've", "she will have", txt)
    txt = re.sub("she's", "she is", txt)
    txt = re.sub("should've", "should have", txt)
    txt = re.sub("shouldn't", "should not", txt)
    txt = re.sub("shouldn't've", "should not have", txt)
    txt = re.sub("so've", "so have", txt)
    txt = re.sub("so's", "so as", txt)
    txt = re.sub("this's", "this is", txt)
    txt = re.sub("that'd", "that would", txt)
    txt = re.sub("that'd've", "that would have", txt)
    txt = re.sub("that's", "that is", txt)
    txt = re.sub("there'd", "there would", txt)
    txt = re.sub("there'd've", "there would have", txt)
    txt = re.sub("there's", "there is", txt)
    txt = re.sub("here's", "here is", txt)
    txt = re.sub("they'd", "they would", txt)
    txt = re.sub("they'd've", "they would have", txt)
    txt = re.sub("they'll", "they will", txt)
    txt = re.sub("they'll've", "they will have", txt)
    txt = re.sub("they're", "they are", txt)
    txt = re.sub("they've", "they have", txt)
    txt = re.sub("to've", "to have", txt)
    txt = re.sub("wasn't", "was not", txt)
    txt = re.sub("we'd", "we would", txt)
    txt = re.sub("we'd've", "we would have", txt)
    txt = re.sub("we'll", "we will", txt)
    txt = re.sub("we'll've", "we will have", txt)
    txt = re.sub("we've", "we have", txt)
    txt = re.sub("weren't", "were not", txt)
    txt = re.sub("what'll", "what will", txt)
    txt = re.sub("what'll've", "what will have", txt)
    txt = re.sub("what're", "what are", txt)
    txt = re.sub("what's", "what is", txt)
    txt = re.sub("what've", "what have", txt)
    txt = re.sub("when's", "when is", txt)
    txt = re.sub("when've", "when have", txt)
    txt = re.sub("where'd", "where did", txt)
    txt = re.sub("where's", "where is", txt)
    txt = re.sub("where've", "where have", txt)
    txt = re.sub("who'll", "who will", txt)
    txt = re.sub("who'll've", "who will have", txt)
    txt = re.sub("who's", "who is", txt)
    txt = re.sub("who've", "who have", txt)
    txt = re.sub("why's", "why is", txt)
    txt = re.sub("why've", "why have", txt)
    txt = re.sub("will've", "will have", txt)
    txt = re.sub("won't", "will not", txt)
    txt = re.sub("won't've", "will not have", txt)
    txt = re.sub("would've", "would have", txt)
    txt = re.sub("wouldn't", "would not", txt)
    txt = re.sub("wouldn't've", "would not have", txt)
    txt = re.sub("y'all", "you all", txt)
    txt = re.sub("y'all'd", "you all would", txt)
    txt = re.sub("y'all'd've", "you all would have", txt)
    txt = re.sub("y'all're", "you all are", txt)
    txt = re.sub("y'all've", "you all have", txt)
    txt = re.sub("you'd", "you would", txt)
    txt = re.sub("you'd've", "you would have", txt)
    txt = re.sub("you'll", "you will", txt)
    txt = re.sub("you'll've", "you will have", txt)
    txt = re.sub("you're", "you are", txt)
    txt = re.sub("you've", "you have", txt)
    return txt

# To lower
def toLowCase(txt):
    txt = txt.lower()
    return txt

In [ ]:
train_df.question_text = train_df.question_text.apply(clean_text)
train_df.question_text = train_df.question_text.apply(clean_numbers)
train_df.question_text = train_df.question_text.apply(correct)
train_df.question_text = train_df.question_text.apply(toLowCase)
train_df.head(7)

In [ ]:
test_df.question_text = test_df.question_text.apply(clean_text)
test_df.question_text = test_df.question_text.apply(clean_numbers)
test_df.question_text = test_df.question_text.apply(correct)
test_df.question_text = test_df.question_text.apply(toLowCase)
test_df.head(7)

In [ ]:
val_df.question_text = val_df.question_text.apply(clean_text)
val_df.question_text = val_df.question_text.apply(clean_numbers)
val_df.question_text = val_df.question_text.apply(correct)
val_df.question_text = val_df.question_text.apply(toLowCase)
val_df.head(7)

# Tokenisation

In [ ]:
# Tokenizing the vocabulary
#Tokenization is the process of dividing text into a set of meaningful pieces. These pieces are called tokens. For example, we can divide a chunk of text into words, or we can divide it into sentences.
#Now we will tokenize our sentences and create the vocabulary. You could set a max. number for the word count in your vocalubary 
# how many unique words to use
max_features = 95000

## Specifying how many words we're going to use and filtering the once that we won't use
tokenizer = Tokenizer(num_words = max_features, filters='')

## This will give you a list of integer sequences encoding the words in your sentence
#fit_on_texts fits on sentences when list of sentences is passed to fit_on_texts() function. 
#ie - fit_on_texts( [ sent1, sent2, sent3,....sentN ] )
tokenizer.fit_on_texts(list(train_df.question_text))
tokenizer.fit_on_texts(list(test_df.question_text))
tokenizer.fit_on_texts(list(val_df.question_text))

# (texts_to_sequences) Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
train_X = tokenizer.texts_to_sequences(train_df.question_text)
test_X = tokenizer.texts_to_sequences(test_df.question_text)
val_X = tokenizer.texts_to_sequences(val_df.question_text)

train_Y = train_df.target.values
val_Y = val_df.target.values

# Create vocabulary from all words
# vocabulary is used to create the tokenized input sentences. Finally, the tokens of these sentences are passed as inputs to the model
vocabulary = tokenizer.word_index
print(train_X[:3])

# Padding

In [ ]:
# Padding sequences 
# Rememeber : The maximum length of our input can not be greater than 100 so we need to pad the incoming sequences at 100
# maximum number of words in a question. This will be our input size
maxlen = 50
train_X = pad_sequences(train_X, maxlen = maxlen, padding='post')
test_X = pad_sequences(test_X, maxlen = maxlen, padding='post')
val_X = pad_sequences(val_X, maxlen = maxlen, padding='post')
print(train_X[:3])

# Embedding

In [ ]:
!unzip ../input/quora-insincere-questions-classification/embeddings.zip

In [ ]:
news_path = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embedding_data = KeyedVectors.load_word2vec_format(news_path, binary=True)

In [ ]:
""""# Load embedding
embedding_data = KeyedVectors.load("../input/gensim-embeddings-dataset/paragram_300_sl999.gensim")""""

In [ ]:
import operator 

def check_coverage(vocab, embedding_item):

    in_vocab = {}
    out_of_vocab = {}
    for word in vocab:
        try:
            in_vocab[word] = embedding_item[word]
        except:
            out_of_vocab[word] = vocab[word]
            pass

    print(f"{len(in_vocab) / len(vocab):.0%} of vocabulary is covered")
    
    sorted_x = sorted(out_of_vocab.items(), key=operator.itemgetter(1))[::-1]
    
    return sorted_x

oov = check_coverage(vocabulary, embedding_data)

In [ ]:
oov[:20]

In [ ]:
def get_embedding_matrix(vocab, embedding_index):

    # Initialize embedding matrix
    embedding_matrix = np.zeros((len(vocab) + 1, 300))

    print(embedding_matrix)
    
    print("-----------------------------------------------------")
    
    for word, i in vocab.items(): 
        if word in embedding_index:
            embedding_matrix[i] = embedding_index[word]
            
    print(embedding_matrix)

    return embedding_matrix

embedding_matrix = get_embedding_matrix(vocabulary, embedding_data)

In [ ]:
embedding_matrix[:1]

## Model

In [ ]:
# building our model

def model_builder():
    inp = Input(shape=(maxlen,))
    x = Embedding(len(embedding_matrix), 300)(inp)
    
    #Bidirectional LSTMs are an extension of traditional LSTMs that can improve model performance on sequence classification problems.
    #In problems where all timesteps of the input sequence are available,
    #Bidirectional LSTMs train two instead of one LSTMs on the input sequence
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    
    #That return sequences return the hidden state output for each input time step.
    x = GlobalAveragePooling1D()(x)
    
    #Activation functions are mathematical equations that determine the output of a neural network
    #rerlu filtering the negative elements
    x = Dense(16, activation = 'relu')(x)
    
    #drop 10% of the neurons
    x = Dropout(0.1)(x)
    
    #transform the input into values between 0 and 1
    x = Dense(1, activation = 'sigmoid')(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

model = model_builder()
    ## to prevent over-fitting
## A fully connected layer occupies most of the parameters, and hence,
## neurons develop co-dependency amongst each other during training
## which curbs the individual power of each neuron leading to over-fitting of training data.

In [ ]:
e_stop = callbacks.EarlyStopping(mode='min', patience=3, restore_best_weights=True)
model_checkp = callbacks.ModelCheckpoint('./w.h5', save_best_only=True, save_weights_only=True)

In [ ]:
model.fit(train_X, train_Y, batch_size=1024, epochs=1, callbacks=[e_stop, model_checkp ], 
          validation_data=(val_X, val_Y))

In [ ]:
prediction = model.predict([test_X], batch_size=1024, verbose=1)

In [ ]:
def pp(y_pred):
    for k in range(len(y_pred)):
        if y_pred[k]>0.5:
            y_pred[k] = 1
        else:
            y_pred[k] = 0
    return y_pred

In [ ]:
prediction=pp(prediction)

In [ ]:
sub_df = pd.DataFrame()
sub_df['qid'] = test_df.qid.values
sub_df['prediction'] = prediction.astype(int)
sub_df.to_csv("submission.csv", columns=['qid', 'prediction'], index_label=False, index=False)
pd.read_csv('./submission.csv')